In [1]:
import numpy as np
import pandas as pd
from Drive import *
import AnalysisDataCleaner
import SupportFunctions

In [2]:
class Analysis:
    def __init__(self):
        self.fileDownloader()
        self.data=self.data_importer()
        self.data=AnalysisDataCleaner.data_cleaner(self.data)
        self.months=['January','February','March','April','May',\
                     'June','July','August','September','October',\
                     'November','December']
        
    def fileDownloader(self):
        """Downloads Student Gradebook.xlsx from GDrive"""
        try:
            auth=GDriveAuthenticator.GDriveAuthenticator('Drive\\client_secret.json')
            drive=Gdrive.Gdrive(auth.authenticate())
            drive.download()
        except:
            print('Problem in DRIVE API usage')
            raise
            
    def data_importer(self):
        """imports the data and merges into one dataframe and returns it"""
        excel_sheet=pd.read_excel('Student Gradebook.xlsx',sheet_name=None,usecols=range(0,10))
        list_data=[]
        for i in excel_sheet:
            list_data.append(excel_sheet[i])
        #data_july=excel_sheet['July'].copy()
        #data_august=excel_sheet['August'].copy()
        data=pd.concat(list_data).reset_index().drop(columns=['index'])
        #del data_july
        #del data_august
        return data
    
    def total_marks_leaderboard(self,month_name=None):
        """Returns a leaderboard of students containing data such as :- total, task, myday, knowledge sharing marks and
        count of late submissions and task won.
        Arguements:-
        :month_name - [Default: None] if any other value then returns leaderboard for the specified month. Make sure first
        letter of month is capital.

        Returns:
        tuple:- (dataframe object,dataframe with hidden indexes)"""
        data=self.data
        if month_name==None:
            total_marks=data.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            total_marks.rename(columns={'Points':'Marks'})
            total_marks['Total Marks']=total_marks['Student'].apply(SupportFunctions.get_highest_marks,args=[data])
            total_marks['Task Marks']=total_marks['Student'].apply(SupportFunctions.get_task,args=[data])
            total_marks['MyDay Marks']=total_marks['Student'].apply(SupportFunctions.get_myday,args=[data])
            total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(SupportFunctions.get_gyan,args=[data])
            total_marks['Late Submissions']=total_marks['Student'].apply(SupportFunctions.get_count_late_submission,args=[data])
            total_marks['Tasks Won']=total_marks['Student'].apply(SupportFunctions.get_count_task_won,args=[data])
            return tuple((total_marks,total_marks.style.hide_index()))
        
        elif month_name in self.months:
            temp=data[data['Month']==month_name]
            total_marks=data[data['Month']==month_name].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            total_marks.rename(columns={'Points':'Marks'})
            total_marks['Total Marks']=total_marks['Student'].apply(SupportFunctions.get_highest_marks,args=[temp,month_name])
            total_marks['Task Marks']=total_marks['Student'].apply(SupportFunctions.get_task,args=[temp,month_name])
            total_marks['MyDay Marks']=total_marks['Student'].apply(SupportFunctions.get_myday,args=[temp,month_name])
            total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(SupportFunctions.get_gyan,args=[temp,month_name])
            total_marks['Late Submissions']=total_marks['Student'].apply(SupportFunctions.get_count_late_submission,args=[temp,month_name])
            total_marks['Tasks Won']=total_marks['Student'].apply(SupportFunctions.get_count_task_won,args=[temp,month_name])
            return tuple((total_marks,total_marks.style.hide_index()))
        
        else:
            raise ValueError('month_name has wrong spelling supplied')
            
    def student_task_marks(self,name=None,month_name=None):
        """Returns the task and points of the given student name in DataFrame
        Arguments:
            data: dataframe from which marks need to be extracted
            name: name of the student, only the first name and first letter capital.
            monthWise: default: False
                        set to True if dataframe needed only for a particular month
            month_name: default None.
                        if monthWise True and month_name is name of the month then
                        Dataframe of that particular month is given.

        Returns:
            tuple:- (dataframe object,dataframe with hidden indexes)
        """
        data=self.data
        if name==None:
            raise ValueError('Input a Name')
            
        if month_name!=None:
            if month_name in self.months:
                task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') &\
                                     (data['Student']==name) & \
                                     (data['Month']==month_name)]
                
            else:
                raise ValueError('Wrong month spelling given')
                
        elif month_name==None:
            task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') &\
                                 (data['Student']==name)]
        task_marks=task_marks[['Date','Month','Task','Student','Points','Total','Late Submission','Task Winner']]
        task_marks['Late Submission'].replace([0,1],['No','Yes'],inplace=True)
        task_marks['Task Winner'].replace([0,1],['No','Yes'],inplace=True)
        return tuple((task_marks,task_marks.style.hide_index()))
    
    def student_myday(self,name=None,month_name=None):
        """Gets the myday marks of the given student
        Arguements:
        name - name of the student
        month_name - [Default: None] name of month to check the dataframe for.

        Returns:
        tuple:- (dataframe object,dataframe with hidden indexes)"""
        data=self.data
        myday='AjKyaUkhada'
        if name==None:
            raise ValueError('Input a name')
        if month_name==None:
            myday_marks=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].\
                sum().sort_values(ascending=False).reset_index()
            myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
            myday_count=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].\
                count().sort_values(ascending=False).reset_index()
            myday_count.rename(columns={'Points':'Number of days'},inplace=True)
            myday_data=myday_marks.merge(myday_count,on='Student')
            return tuple((myday_data,myday_data.style.hide_index()))
        elif month_name in self.months:
            myday_marks=data[(data['Month']==month_name) & (data['Task']==myday) &\
                             (data['Student']==name)].groupby('Student')['Points'].\
                            sum().sort_values(ascending=False).reset_index()
            myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
            myday_count=data[(data['Month']==month_name) & (data['Task']==myday) & \
                             (data['Student']==name)].groupby('Student')['Points'].\
                            count().sort_values(ascending=False).reset_index()
            myday_count.rename(columns={'Points':'Number of days'},inplace=True)
            myday_data=myday_marks.merge(myday_count,on='Student')
            return tuple((myday_data,myday_data.style.hide_index()))

    def student_knowledge_marks(self,name=None,month_name=None):
        """Gets the knowledge sharing marks of the given student
        Arguements:
        name - name of the student
        month_name - [Default: None] name of month to check the dataframe for.

        Returns:
        tuple:- (dataframe object, dataframe with hidden indexes)
        """
        data=self.data
        knowledge='Knowledge Sharing'
        if name==None:
            raise ValueError('Input a name')
        if month_name==None:
            knowledge_marks=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
            knowledge_count=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
            knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
            knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
            return tuple((knowledge_data,knowledge_data.style.hide_index()))
        elif month_name in self.months:
            knowledge_marks=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
            knowledge_count=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
            knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
            knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
            return tuple((knowledge_data,knowledge_data.style.hide_index()))
    
    def get_rank(self,name=None,month_name=None):
        """Get the rank of the student, either from the global leaderboard, or from a months leaderboard (if month name
        given)
        :name - name of the student
        :month_name - [Default:None] enter a month to run the function on.

        Returns:
        rank of the student in string format"""
        data=self.data
        
        if name==None:
            raise ValueError('Input a name')

        if month_name==None:
            leaderboard=self.total_marks_leaderboard()[0]
            leaderboard=leaderboard[leaderboard['Student']==name].index
            if len(leaderboard)>0:
                return str(leaderboard[0]+1) #index starts from zero, so index+1 is rank
            else:
                raise Exception('Name wrong.')
        elif month_name in self.months:
            leaderboard=self.total_marks_leaderboard(month_name=month_name)[0]
            leaderboard=leaderboard[leaderboard['Student']==name].index
            if len(leaderboard)>0:
                return str(leaderboard[0]+1) #index starts from zero, so index+1 is rank
            else:
                raise Exception('Wrong name.')
        else:
            raise Exception('Please check spelling of month and try again')
            
    def get_total_marks(self,name=None,month_name=None):
        data=self.data
        if name==None:
            raise ValueError('input name')
        if month_name==None:
            leaderboard=self.total_marks_leaderboard()[0]
            leaderboard=leaderboard[leaderboard['Student']==name]['Points'].values
            if len(leaderboard)>0:
                return str(leaderboard[0]) # returning total marks of the student as string
            else:
                raise Exception('Name wrong.')
        elif month_name in self.months:
            leaderboard=self.total_marks_leaderboard(month_name=month_name)[0]
            leaderboard=leaderboard[leaderboard['Student']==name]['Points'].values
            if len(leaderboard)>0:
                return str(leaderboard[0]) # returning total marks of the student as string
            else:
                raise Exception('Wrong name.')
        else:
            raise Exception('Please check spelling of month and try again')


In [3]:
a=Analysis()

Download. 100


In [4]:
a.total_marks_leaderboard()[0]

Student  Points  Total Marks  Task Marks  MyDay Marks  \
0          Kunal   354.5          378       241.5        107.0   
1       Ujjainee   348.0          380       233.0        105.0   
2         Shakib   343.0          376       232.0        101.0   
3        Surabhi   339.0          373       231.0        105.0   
4        Purbita   337.0          371       231.0         97.0   
5         Vishal   328.5          365       228.5         98.0   
6      Chandrima   327.0          379       213.0        103.0   
7         Sonali   326.5          367       224.5         92.0   
8   Siddhishikha   326.5          375       226.5         92.0   
9          Dipam   326.5          374       217.5        105.0   
10         Durga   326.0          379       222.0         99.0   
11       Sharika   315.0          352       218.0         95.0   
12        Apurwa   308.0          354       219.0         89.0   
13       Kaushal   287.0          342       200.0         84.0   
14     Swaastick   273.0          312       206.0         67.0   
15       Roumyak   267.5          314       218.5         49.0   
16      Shantanu   263.0          293       225.0         38.0   
17       Prasoon   252.0          293       184.0         62.0   
18          Arya   207.5          244       183.5         24.0   
19        Bhavna   205.0          245       170.0         30.0   
20        Anjali   152.0          179       143.0          9.0   
21         Wrick    60.5           89        46.5         14.0   
22     Sudhanshu     8.5           12         6.5          2.0   

    Knowledge Sharing Marks  Late Submissions  Tasks Won  
0                       6.0                 5          4  
1                      10.0                 4          0  
2                      10.0                 4          3  
3                       3.0                 4          1  
4                       9.0                 6          2  
5                       2.0                 2          2  
6                      11.0                 6          3  
7                      10.0                 5          2  
8                       8.0                 8          0  
9                       4.0                 2          1  
10                      5.0                 6          1  
11                      2.0                 7          2  
12                      0.0                 5          3  
13                      3.0                 6          1  
14                      0.0                12          2  
15                      0.0                 6          2  
16                      0.0                 7          2  
17                      6.0                 7          0  
18                      0.0                 9          2  
19                      5.0                 3          0  
20                      0.0                 1          1  
21                      0.0                 3          0  
22                      0.0                 1          0

In [5]:
a.total_marks_leaderboard(month_name='September')[0]

Student  Points  Total Marks  Task Marks  MyDay Marks  \
0          Durga   125.5          136        99.5         25.0   
1          Dipam   125.0          128        92.0         29.0   
2   Siddhishikha   124.5          133        96.5         24.0   
3         Shakib   123.5          128        90.5         29.0   
4         Sonali   122.0          129        88.0         30.0   
5      Chandrima   122.0          129        88.0         29.0   
6          Kunal   122.0          126        91.0         29.0   
7        Purbita   120.5          127        88.5         27.0   
8       Ujjainee   119.5          128        86.5         29.0   
9        Surabhi   117.5          125        87.5         29.0   
10        Apurwa   115.5          122        88.5         27.0   
11       Sharika   112.5          118        89.5         23.0   
12      Shantanu   111.5          117        89.5         22.0   
13        Vishal   111.0          117        89.0         22.0   
14       Kaushal    95.0          104        76.0         18.0   
15       Roumyak    93.0          102        86.0          7.0   
16     Swaastick    90.5           94        71.5         19.0   
17       Prasoon    89.5           97        67.5         18.0   
18        Bhavna    84.5           89        55.5         24.0   
19          Arya    60.0           64        46.0         14.0   
20        Anjali    46.5           49        37.5          9.0   

    Knowledge Sharing Marks  Late Submissions  Tasks Won  
0                       1.0                 3          0  
1                       4.0                 2          1  
2                       4.0                 4          0  
3                       4.0                 2          1  
4                       4.0                 2          1  
5                       5.0                 3          2  
6                       2.0                 2          2  
7                       5.0                 3          0  
8                       4.0                 3          0  
9                       1.0                 3          0  
10                      0.0                 2          2  
11                      0.0                 3          1  
12                      0.0                 3          1  
13                      0.0                 2          1  
14                      1.0                 2          0  
15                      0.0                 4          1  
16                      0.0                 3          1  
17                      4.0                 3          0  
18                      5.0                 2          0  
19                      0.0                 1          0  
20                      0.0                 0          1

In [6]:
a.student_task_marks('Kunal')[0]

Date      Month                                      Task Student  \
5       22       July            Linkedin Profile create/update   Kunal   
45      23       July  Blog on Machine Learning and it's impact   Kunal   
155     30       July    Blog on Different data based job roles   Kunal   
195     31       July                       Portfolio home page   Kunal   
246      1     August                              Video Resume   Kunal   
284      8      March                             Online Resume   Kunal   
316      8      April                    Blog and Project Pages   Kunal   
523     13     August                   Github Profile Creation   Kunal   
568     14     August                            PPT Submission   Kunal   
766     23     August             Blog on Data Analysis Process   Kunal   
1162    11  September                              Web Scraping   Kunal   
1296    17  September                             Data from API   Kunal   
1328    17  September                          Batsman V Bowler   Kunal   
1375    17  September                     Start/Country Dataset   Kunal   

      Points  Total Late Submission Task Winner  
5        8.0     10              No          No  
45      21.0     25              No          No  
155     21.5     25             Yes         Yes  
195     12.0     15             Yes          No  
246     20.0     20              No          No  
284     18.0     20              No          No  
316      9.5     10              No          No  
523      9.5     10              No         Yes  
568      5.0      5             Yes          No  
766     26.0     30              No          No  
1162    10.0     10              No         Yes  
1296    15.0     15             Yes          No  
1328    20.0     20              No         Yes  
1375    46.0     50             Yes          No

In [7]:
a.student_task_marks('Kunal','September')[0]

Date      Month                   Task Student  Points  Total  \
1162    11  September           Web Scraping   Kunal    10.0     10   
1296    17  September          Data from API   Kunal    15.0     15   
1328    17  September       Batsman V Bowler   Kunal    20.0     20   
1375    17  September  Start/Country Dataset   Kunal    46.0     50   

     Late Submission Task Winner  
1162              No         Yes  
1296             Yes          No  
1328              No         Yes  
1375             Yes          No

In [8]:
a.student_myday('Kunal')[0]

Student  Myday Marks  Number of days
0   Kunal        107.0              59

In [9]:
a.student_myday('Kunal',month_name='September')[0]

Student  Myday Marks  Number of days
0   Kunal         29.0              20

In [10]:
a.student_knowledge_marks('Kunal')[0]

Student  Knowledge Sharing Marks  Number of days
0   Kunal                      6.0               3

In [11]:
a.student_knowledge_marks('Kunal',month_name='August')[0]

Student  Knowledge Sharing Marks  Number of days
0   Kunal                      4.0               2

In [12]:
a.data

id                            Task  Date            Module  \
0      1  Linkedin Profile create/update    22  Profile Building   
1      1  Linkedin Profile create/update    22  Profile Building   
2      1  Linkedin Profile create/update    22  Profile Building   
3      1  Linkedin Profile create/update    22  Profile Building   
4      1  Linkedin Profile create/update    22  Profile Building   
5      1  Linkedin Profile create/update    22  Profile Building   
6      1  Linkedin Profile create/update    22  Profile Building   
7      1  Linkedin Profile create/update    22  Profile Building   
8      1  Linkedin Profile create/update    22  Profile Building   
9      1  Linkedin Profile create/update    22  Profile Building   
10     1  Linkedin Profile create/update    22  Profile Building   
11     1  Linkedin Profile create/update    22  Profile Building   
12     1  Linkedin Profile create/update    22  Profile Building   
13     1  Linkedin Profile create/update    22  Profile Building   
14     1  Linkedin Profile create/update    22  Profile Building   
15     1  Linkedin Profile create/update    22  Profile Building   
16     1  Linkedin Profile create/update    22  Profile Building   
17     1  Linkedin Profile create/update    22  Profile Building   
18     1  Linkedin Profile create/update    22  Profile Building   
19     1  Linkedin Profile create/update    22  Profile Building   
20     1  Linkedin Profile create/update    22  Profile Building   
21     1  Linkedin Profile create/update    22  Profile Building   
22     1  Linkedin Profile create/update    22  Profile Building   
23     0                     AjKyaUkhada    22            Ritual   
24     0                     AjKyaUkhada    22            Ritual   
25     0                     AjKyaUkhada    22            Ritual   
26     0                     AjKyaUkhada    22            Ritual   
27     0                     AjKyaUkhada    22            Ritual   
28     0                     AjKyaUkhada    22            Ritual   
29     0                     AjKyaUkhada    22            Ritual   
...   ..                             ...   ...               ...   
1408   0               Knowledge Sharing    19            Ritual   
1409   0                     AjKyaUkhada    19            Ritual   
1410   0                     AjKyaUkhada    19            Ritual   
1411   0                     AjKyaUkhada    19            Ritual   
1412   0               Knowledge Sharing    19            Ritual   
1413   0                     AjKyaUkhada    19            Ritual   
1415   0                     AjKyaUkhada    19            Ritual   
1416   0                     AjKyaUkhada    19            Ritual   
1418   0                     AjKyaUkhada    20            Ritual   
1419   0                     AjKyaUkhada    20            Ritual   
1420   0                     AjKyaUkhada    20            Ritual   
1421   0                     AjKyaUkhada    20            Ritual   
1422   0               Knowledge Sharing    20            Ritual   
1423   0                     AjKyaUkhada    20            Ritual   
1424   0                     AjKyaUkhada    20            Ritual   
1425   0                     AjKyaUkhada    20            Ritual   
1426   0                     AjKyaUkhada    20            Ritual   
1427   0                     AjKyaUkhada    20            Ritual   
1428   0                     AjKyaUkhada    20            Ritual   
1429   0                     AjKyaUkhada    20            Ritual   
1430   0                     AjKyaUkhada    20            Ritual   
1431   0                     AjKyaUkhada    20            Ritual   
1432   0                     AjKyaUkhada    20            Ritual   
1433   0                     AjKyaUkhada    20            Ritual   
1434   0                     AjKyaUkhada    21            Ritual   
1435   0                     AjKyaUkhada    21            Ritual   
1436   0                     AjKyaUkhada    21            Ritu

In [13]:
a.get_rank(name='Kunal',month_name='September')

'7'